In [1]:
import pandas as pd
import tensorflow as tf
import scipy as sc
import cv2
import numpy as np
import os
import json
from base64 import b64encode
from typing import (
    Any,
    List,
    Tuple,
)

import align.detect_face    # type: ignore
import facenet              # type: ignore

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
base_path = "D:/NNA/"
data_path = base_path + "data/"
facenet_path = base_path + "node-facenet/"

In [3]:
class MtcnnBridge():
    """
    MTCNN Face Alignment
    """
    def __init__(self) -> None:
        self.graph = self.session = None            # type: Any
        self.pnet = self.rnet = self.onet = None    # type: Any

        self.minsize = 20                   # minimum size of face
        self.threshold = [0.6, 0.7, 0.7]    # three steps's threshold
        self.factor = 0.709                 # scale factor

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                self.pnet, self.rnet, self.onet = \
                    align.detect_face.create_mtcnn(self.session, None)

    def align(
            self,
            image: np.ndarray,
    ) -> Tuple[List[Any], List[Any]]:
        """ doc """
        

        bounding_boxes, landmarks = align.detect_face.detect_face(
            image,   # get rid of alpha channel(if any)
            self.minsize,
            self.pnet,
            self.rnet,
            self.onet,
            self.threshold,
            self.factor,
        )

        return bounding_boxes.tolist(), landmarks.tolist()

In [4]:
class FacenetBridge(object):
    """
    Bridge of Facenet
    """
    FACENET_MODEL = None   # type: str

    def __init__(self) -> None:
        self.graph = self.session = None        # type: Any

        self.placeholder_input = None           # type: Any
        self.placeholder_phase_train = None     # type: Any
        self.placeholder_embeddings = None      # type: Any

        self.FACENET_MODEL = FacenetBridge.get_model_path()

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                model_dir = os.path.expanduser(self.FACENET_MODEL)
                meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
                saver = tf.train.import_meta_graph(
                    os.path.join(model_dir, meta_file),
                )
                saver.restore(
                    tf.get_default_session(),
                    os.path.join(model_dir, ckpt_file),
                )
                # facenet.load_model(self.FACENET_MODEL)

        self.placeholder_input = self.graph.get_tensor_by_name('input:0')
        self.placeholder_phase_train = \
            self.graph.get_tensor_by_name('phase_train:0')
        self.placeholder_embeddings = \
            self.graph.get_tensor_by_name('embeddings:0')

    @staticmethod
    def get_model_path() -> str:
        """
        Get facenet model path from package.json
        """
        try:
            model_path = os.environ['FACENET_MODEL']    # type: str
            return model_path
        except KeyError:
            pass

        
        module_root = facenet_path

        package = os.path.join(module_root, 'package.json')
        with open(package) as data:
            package_json = json.load(data)

        python_facenet_model_path = \
            package_json['facenet']['env']['PYTHON_FACENET_MODEL_PATH']
        model_path = os.path.join(module_root, python_facenet_model_path)

        if not os.path.exists(model_path):
            raise FileNotFoundError(
                errno.ENOENT,
                os.strerror(errno.ENOENT),
                model_path
            )

        return model_path

    def embedding(
            self,
            image: np.ndarray
    ) -> List[float]:
        """
        Get embedding
        """
        #image = base64_to_image(image_base64, row, col, depth)

        if image.ndim == 2:
            image = facenet.to_rgb(image)

        # get rid of Alpha Channel from PNG(if any) and prewhiten
        image = facenet.prewhiten(image[:, :, 0:3])

        feed_dict = {
            self.placeholder_input:         image[np.newaxis, :],
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        # Return the only row
        return embeddings[0].tolist()

In [5]:
showFullImage = False
showResizedImage = False


CSV_COLUMN_NAMES = ['numb', 'first_name', 'last_name', 'file_name']

emp_list = pd.read_csv(data_path + "emp.csv", names=CSV_COLUMN_NAMES, header=0)
#print(emp_list.head())


mtcnn = MtcnnBridge()
mtcnn.init()


face = FacenetBridge()
face.init()

print("Start!!!")

#emp_list = emp_list.head(5)

records = []

for emp in emp_list.values:
    
    rec = {
        "name": emp[1] + " " + emp[2],
        "first_name": emp[1],
        "last_name": emp[2],
        "file_name": emp[3],
        "lvl": 0,
        "embedding": []
    }
    
    if emp[3] == "noPhoto.png":
        continue
    
    image_path = data_path + "images/" + emp[3]
    image = pl.imread(fname=image_path)
    res = mtcnn.align(image)    

    pos = res[0][0]
    x = int(pos[0])
    y = int(pos[1])
    w = int(pos[2])
    h = int(pos[3])
    
    img = image[y:h, x:w, 0:3]
        
    print(rec["name"])
    
    if showFullImage:        
        pl.figure()
        pl.imshow(image)
        currentAxis = pl.gca()
        currentAxis.add_patch(pl.Rectangle((x, y), w - x, h - y, ec='r', fc='none'))
        pl.show()
        
    img = cv2.resize(img, dsize=(160, 160), interpolation=cv2.INTER_CUBIC)
    emb = face.embedding(img)
        
    rec["embedding"] = emb
    
    records.append(rec)
    
    if showResizedImage:
        print(rec["name"])
        pl.figure()
        pl.imshow(img)
        pl.show()

with open(data_path + 'data.json', 'w+') as outfile:
    json.dump(records, outfile)

print("Ready!!!")


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from D:/NNA/node-facenet/models/model-20170512-110547.ckpt-250000
Start!!!
Andriy Zynych
Lyubomyr Senyuk
Andrian Kolesnyk
Andriy Vesolovskyy
Andriy Yavorskyy
Andriy Demkovych
Andriy Dzyben
Bohdan Bochkov
Bohdan Serednytskyy
Dmytro Andreyev
Dmytro Lutsyuk
Hryhoriy Hasyn
Ihor Stepanushko
Inna Tkachenko
Iryna Sidelnyk
Iryna Vesylivska
Ivan Nahirnyy
Kateryna Hirna
Khrystyna Romanko
Maksym Yakymenko
Maksym Semerak
Maksym Kondakov
Mykola Hev
Nataliya Zynych
Nataliya Soltys
Oleh Kubay
Oleksandr Shumilov
Ostap Hishchak
Pavlo Maltsev
Pavlo Kopytsya
Roman Paranchuk
Roman Skurchak
Roman Dovhanyk
Roman Romanko
Taras Shevchuk
Tetyana Nykonenko
Ulyana Ropyak
Valentyn Zubenko
Vasyl Polevyy
Vitaliy Zinchenko
Volodymyr Bychkovskyy
Yevhen Zavaliy
Yevhen Zaretskyy
Yuriy Bondar
Yuriy Fedorchuk
Yuriy Fadyeyev
Yuriy Demchuk
Pavlo Domanskyy
Kateryna Drobviazko
Khrystyna Lutskiv